###### 図 4: エラーメッセージの例


In [1]:
from config import open_not_exist_file

open_not_exist_file()

FileNotFoundError: [Errno 2] No such file or directory: 'config.txt'

###### 図 5: try 文（try 節・except 節）の記述例


In [2]:
try:
    # 存在しないファイルを開く
    with open("config.txt", "r") as file:
        config = file.read()
        print(f"configファイルを読み込みました。{config=}")
except FileNotFoundError:
    # エラー処理
    config = "default"
    print(f"configファイルが存在しません。デフォルト値を使用します。{config=}")

configファイルが存在しません。デフォルト値を使用します。config='default'


###### 図 6: 複数の except 節の記述例


In [3]:
data = {}
try:
    data["x"] += 1
except KeyError:
    print("キーが存在しません。キーの一覧:", list(data.keys()))
except TypeError:
    print("値の型が数値ではありません。型:", type(data["x"]))
except Exception:
    # Exceptionを指定するとKeyErrorも捕捉可能だが、
    # KeyErrorが先に捕捉しているのでここには到達しない
    print("予期せぬエラーが発生しました。")

キーが存在しません。キーの一覧: []


###### 図 7: タプルを使用した except 節の記述例


In [4]:
data = {}
try:
    data["x"] += 1
# "x" というキーがない場合、もしくは d["x"] が文字列だった場合など
except (KeyError, TypeError):
    print(f"dataに誤りがあります。{data=}")

dataに誤りがあります。data={}


###### 図 8: ロガーにログを記録する例


In [5]:
import logging

# logging.DEBUG以上のログを出力する設定
logging.basicConfig(level=logging.DEBUG)

logging.debug("DEBUGメッセージ")
logging.info("INFOメッセージ")
# 文字列以外(辞書など)も記録できる
# 文字列以外の場合は、str()で文字列に変換して出力される
error_code = 1000
logging.warning({"WARNINGメッセージ": error_code})
error_object = ValueError("error object")
logging.error({"ERRORメッセージ": error_object})
# フォーマット文字列も使用可能
logging.critical(f"CRITICALメッセージ {error_code=}")

DEBUG:root:DEBUGメッセージ
INFO:root:INFOメッセージ
ERROR:root:{'ERRORメッセージ': ValueError('error object')}
CRITICAL:root:CRITICALメッセージ error_code=1000


###### 図 9: ロガーの設定例


In [6]:
logging.basicConfig(
    filename="example.log",  # example.logファイルに記録
    encoding="utf-8",  # utf-8で記録
    level=logging.DEBUG,  # 重要度がlogging.DEBUG以上のイベントを記録
)

###### 図 10: 捕捉した例外をロガーに記録する例


In [7]:
data = {}
try:
    data["x"] += 1
except KeyError:
    logging.error(f"xをインクリメントできませんでした。{data=}", exc_info=True)

ERROR:root:xをインクリメントできませんでした。data={}
Traceback (most recent call last):
  File "/tmp/ipykernel_21603/1648662037.py", line 3, in <module>
    data["x"] += 1
    ~~~~^^^^^
KeyError: 'x'


###### 図 11: 例外を捕捉し何もしない例


In [8]:
import time

# 成功・失敗の制御フラグ
attempt = 0


def connect_to_server() -> None:
    """サーバーに接続を試みる(シミュレーション)"""
    global attempt
    attempt += 1
    if attempt == 1:  # 1回目は失敗
        raise ConnectionError("接続に失敗しました。")
    print("サーバーへの接続に成功しました。")

In [ ]:
try:
    # 例外が発生する可能性がある処理(サーバに接続)
    connect_to_server()
except ConnectionError as e:
    print(f"{e} リトライします...")
    time.sleep(1)
    connect_to_server()

接続に失敗しました。 リトライします...
サーバーへの接続に成功しました。


###### リスト 2: contextlib.suppress の記述例


In [10]:
from contextlib import suppress

d = {}
with suppress(KeyError):
    d["x"] += 1

###### 図 12: 例外を捕捉しノートを追加して再送出する、例外の引数とノートを確認する例


In [11]:
input_number = -1
try:
    if input_number < 0:
        raise ValueError("無効な値が入力されました")
except ValueError as e:
    # ノートを追加
    e.add_note("原因: 値が負数です")
    e.add_note("対処法: 正の値を入力してください")
    # argsには引数で渡された値が格納される
    print("引数 (args):", e.args)
    # __notes__にはadd_noteで追加した捕捉情報が格納される
    print("ノート (__notes__):", e.__notes__)
    raise

引数 (args): ('無効な値が入力されました',)
ノート (__notes__): ['原因: 値が負数です', '対処法: 正の値を入力してください']


ValueError: 無効な値が入力されました

###### 図 13: 例外を捕捉しさらに例外を上乗せする例(from eがある場合)

In [12]:
import json
from collections.abc import Mapping


def load_settings(settings_file: str) -> Mapping[str, str]:
    """設定ファイル(json)を読み込む"""
    with open(settings_file, "r") as file:
        return json.load(file)

In [13]:
settings = load_settings("settings.json")  # { "dummy_key": "dummy_value" }
try:
    setting_value = settings["setting_key"]
except KeyError as e:
    raise ValueError("設定ファイルが壊れています。") from e

ValueError: 設定ファイルが壊れています。

###### 参考: 例外を上乗せした場合のエラーメッセージ例(from e がある場合)


In [14]:
try:
    raise ValueError("最初の例外")
except ValueError as e:
    raise TypeError("上乗せした例外") from e

TypeError: 上乗せした例外

###### 図 14: 例外を上乗せした場合のエラーメッセージ例(from eがない場合)


In [15]:
try:
    raise ValueError("最初の例外")
except ValueError:
    raise TypeError("上乗せした例外")

TypeError: 上乗せした例外

###### 参考: 例外を上乗せした場合の**cause**属性


In [16]:
try:
    try:
        raise ValueError("最初の例外")
    except ValueError as error_1:
        # __cause__ がない例外を再発生させる
        raise TypeError("上乗せした例外") from error_1
except Exception as error_2:  # 上乗せした例外を捕捉
    # 例外に __cause__ が付与されているかを確認
    if error_2.__cause__:
        print(f"{error_2}に__cause__が付いています。")
    else:
        print(f"{type(error_2)}に__cause__は付いていません。")
    print(f"{error_2.__cause__=}")

上乗せした例外に__cause__が付いています。
error_2.__cause__=ValueError('最初の例外')


###### 図 15: 例外を捕捉し例外を差し替える例


In [17]:
items = [-1, -2, -3]
try:
    # ジェネレータ式で最初に条件を満たす要素を取得
    matched_item = next(it for it in items if it > 0)
except StopIteration:
    raise ValueError("該当要素が見つかりませんでした。") from None

ValueError: 該当要素が見つかりませんでした。

###### 図 16: else 節の記述例


In [18]:
try:
    result = 10 / 2  # 例外が発生する可能性のあるコード
except ZeroDivisionError:
    print("0で除算しようとしました。")
    result = 0
else:
    # 例外が発生しなかった場合のみ表示される
    print(f"正常に処理されました。{result=}")

正常に処理されました。result=5.0


###### 図 17: finally 節の記述例


In [19]:
total_count = 0
try:
    result = 10 / 0  # 例外が発生する可能性のあるコード
    total_count += 1
except ZeroDivisionError:
    print("0で除算しようとしました。")
finally:
    # 例外の有無に関わらず実行される
    with open("result.txt", "w") as file:
        file.write(f"{total_count=}")
    print(f"実行回数をファイルに書き込みました。{total_count=}")

0で除算しようとしました。
実行回数をファイルに書き込みました。total_count=0


###### 図 18: 後始末の実行順序


In [20]:
def execution_order_sample() -> None:
    try:
        print("1. try節実行")
    except ValueError as e:
        # エラーが発生しないため、この例ではこのブロックは実行されない
        print(f"2. except節実行: {e}")
    else:
        print("3. else節実行: returnします。")
        return
    finally:
        print("4. finally節実行")


execution_order_sample()

1. try節実行
3. else節実行: returnします。
4. finally節実行


###### 図A: 標準ライブラリと同名のスクリプト実行時のエラーメッセージの例

In [21]:
import sys
import os

# 標準ライブラリと同名の自作スクリプトをimportした状況を再現するため、標準ライブラリではなく自作スクリプトを優先してimportするように変更
# Pythonはsys.pathの最初のエントリから順にモジュールを検索するため、先頭にディレクトリを追加
sys.path.insert(0, os.getcwd())
# すでにrandom・numpyモジュールがインポートされている場合は、削除
sys.modules.pop("random", None)
sys.modules.pop("numpy", None)

In [22]:
# ダミーのrandomモジュールをインポート
import random

random.random()

ダミーのモジュールがimportされました


AttributeError: module 'random' has no attribute 'random' (consider renaming '/workspaces/softwaredesign-202502-python-exception-handling/03_exception_handling/random.py' since it has the same name as the standard library module named 'random' and prevents importing that standard library module)

###### 参考: サードパーティモジュールと同名のスクリプト実行時のエラーメッセージの例

In [23]:
# ダミーのnumpyモジュールをインポート
import numpy as np

np.random.random()

ダミーのモジュールがimportされました


AttributeError: module 'numpy' has no attribute 'random'